# Importing packages

In [1]:
import pandas as pd 
import numpy as np
import datetime

# Setting the option to display the dataframe 
pd.options.display.max_columns = None
pd.options.display.max_rows = 150

# Features in the analysis 

The main features that will be used in the modeling of credit risk are: 

* **earliest_cr_line** - The month the borrower's earliest reported credit line was opened.

* **emp_length** - Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. 

* **term** - number of payments on the loan. Values can be either 36 or 60. 

* **issue_d** - the month which the loan was funded (issued)

* **grade** - loan companies assigned loan grade.

* **sub_grade** - loan companies assigned loan subgrade.

* **home_ownership** - The home ownership status provided by the borrower during registration or obtained from the credit report. The values are: RENT, OWN, MORTGAGE, OTHER.

* **verification_status** - Indicates if income was verified by LC, not verified, or if the income source was verified.

* **purpose** - the purpose of the loan; a category provided by the borrower for the loan request.

* **addr_state** - the state provided by the borrower in the loan application.

* **initial_list_status** - the initial listing status of the loan. Possible values are – W, F.

* **loan_amnt** - the amount of $ applied by the borrower.

* **funded_amnt** - the amount of requested loan that was funded in $.

* **annual_inc** - the self-reported annual income by the borrower.

* **acc_now_delinq** - the amount of accounts which the borrower is a delinquet.

* **total_acc** - The total number of credit lines currently in the borrower's credit file.

* **pub_rec** - Number of derogatory public records.

* **open_acc** - The number of open credit lines in the borrower's credit file.

The feature that we will use for the creation of the $Y$ (also known as the dependant variable) is the loan_status variable. 

* **loan_status** - current status of the loan (as of creation of the data).

In [4]:
columns = [
    'earliest_cr_line',
    'emp_length',
    'term',
    'issue_d',
    'grade',
    'sub_grade',
    'home_ownership',
    'verification_status',
    'loan_status',
    'purpose',
    'addr_state',
    'initial_list_status',
    'loan_amnt',
    'funded_amnt',
    'annual_inc',
    'acc_now_delinq',
    'total_acc',
    'pub_rec',
    'open_acc'
]

# Reading the data 

In [5]:
# Reading the data
d = pd.read_csv(
    'data/Loan_status_2007-2020Q3.gzip', 
    low_memory=False,
    usecols=columns
)

In [6]:
print(f'Shape of the data: {d.shape}')

Shape of the data: (2925493, 19)


In [7]:
# Eyeballing the data
d.head()

,loan_amnt,funded_amnt,term,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,addr_state,earliest_cr_line,open_acc,pub_rec,total_acc,initial_list_status,acc_now_delinq
0,5000.0,5000.0,36 months,B,B2,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,credit_card,AZ,Jan-1985,3.0,0.0,9.0,f,0.0
1,2500.0,2500.0,60 months,C,C4,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,car,GA,Apr-1999,3.0,0.0,4.0,f,0.0
2,2400.0,2400.0,36 months,C,C5,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,small_business,IL,Nov-2001,2.0,0.0,10.0,f,0.0
3,10000.0,10000.0,36 months,C,C1,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,other,CA,Feb-1996,10.0,0.0,37.0,f,0.0
4,3000.0,3000.0,60 months,B,B5,1 year,RENT,80000.0,Source Verified,Dec-2011,Fully Paid,other,OR,Jan-1996,15.0,0.0,38.0,f,0.0


In [8]:
# Seeing the types of the columns to make sure numerics are numeris, 
# categorical variables are categorical, etc.
d.dtypes

loan_amnt              float64
funded_amnt            float64
term                    object
grade                   object
sub_grade               object
emp_length              object
home_ownership          object
annual_inc             float64
verification_status     object
issue_d                 object
loan_status             object
purpose                 object
addr_state              object
earliest_cr_line        object
open_acc               float64
pub_rec                float64
total_acc              float64
initial_list_status     object
acc_now_delinq         float64
dtype: object

# Filling missing data

In [9]:
# Getting the number of rows missing
d.isnull().sum()

loan_amnt                   1
funded_amnt                 1
term                        1
grade                       1
sub_grade                   1
emp_length             205221
home_ownership              1
annual_inc                  5
verification_status         1
issue_d                     1
loan_status                 1
purpose                     1
addr_state                  1
earliest_cr_line           30
open_acc                   30
pub_rec                    30
total_acc                  30
initial_list_status         1
acc_now_delinq             30
dtype: int64

According to the documentation in the data, if the emp_length is missing, it means that there is no employment info. The suggested procedure is to treat this value the same as "< 1 year" value.

In [10]:
d['emp_length'] = d['emp_length'].fillna('< 1 year')

All the other missing rows will be dropped. The amount of rows dropped is very minimal when compared to the total number of rows. 

In [11]:
d.dropna(inplace=True)
d.reset_index(inplace=True, drop=True)

print(f"Shape of data after dropping rows with missing values: {d.shape}")

Shape of data after dropping rows with missing values: (2925463, 19)


# General data preprocesing 

## Categories to numerics

The **term** and **emp_length** features are stored as categorical. We need to convert them first to numeric. 

In [12]:
d['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [13]:
d['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years'],
      dtype=object)

In [14]:
# Cleaning employment length 
d['emp_length_int'] = d['emp_length'].str.replace('\+ years| years| year', '')
d['emp_length_int'] = d['emp_length_int'].str.replace('< 1', str(0))

# Converting the column type to numeric 
d['emp_length_int'] = pd.to_numeric(d['emp_length_int'])

In [15]:
# Cleaning the term column
d['term_int'] = d['term'].str.replace(' months', '')
d['term_int'] = d['term_int'].str.replace(' ', '')

# Converting the column type to numeric 
d['term_int'] = pd.to_numeric(d['term_int'])

## Categories to dates

In [16]:
def convert_to_2digit(x: str) -> datetime.datetime:
    """
    A function to convert 4 digit last part of the date string to 2 digit
    """
    try:
        x = x.split('-')
        month_part = x[0]
        year_part = x[1][-2:]

        return datetime.datetime.strptime(f'{month_part}-{year_part}', '%b-%y')
    except:
        return np.nan

In [17]:
# Inspecting the structure of dates

# Head
d['earliest_cr_line'].unique().tolist()[0:10]

['Jan-1985',
 'Apr-1999',
 'Nov-2001',
 'Feb-1996',
 'Jan-1996',
 'Nov-2004',
 'Jul-2005',
 'Jan-2007',
 'Apr-2004',
 'Sep-2004']

In [18]:
# Tail 
d['earliest_cr_line'].unique().tolist()[-10:]

['Apr-1958',
 'Feb-1945',
 'Mar-1957',
 'Jul-1952',
 'Mar-1933',
 'Aug-1941',
 'Dec-1946',
 'Sep-1951',
 'Feb-1934',
 'Aug-1957']

In [19]:
# Converting to 2 digit years
d['earliest_cr_line_date'] = [convert_to_2digit(x) for x in d['earliest_cr_line']]

# The exact same logic applies to the column issue_d
d['issue_d_date'] = [convert_to_2digit(x) for x in d['issue_d']]

In [20]:
# Calculating the dates between loan issue and first credit line issued 
d['issue_cr_diff'] = d['issue_d_date'] - d['earliest_cr_line_date']

In [21]:
d['issue_cr_diff'].describe()

count                         2925463
mean     5899 days 20:21:30.542796032
std      2962 days 00:57:29.220568992
min             -21915 days +00:00:00
25%                4109 days 00:00:00
50%                5388 days 00:00:00
75%                7366 days 00:00:00
max               18840 days 00:00:00
Name: issue_cr_diff, dtype: object

In [22]:
print(f"Very old credit lines make of {round(d[d['issue_cr_diff'].dt.days<0].shape[0] * 100 / d.shape[0], 2)} % of the data")

Very old credit lines make of 0.2 % of the data


For now, we will drop the very old users. The droping of these rows will have very little influence to the final coefficients.

In [23]:
d = d[d['issue_cr_diff'].dt.days>0]

d.reset_index(inplace=True, drop=True)

By convention, in credit risk modeling, months are prefered vs days. 

In [24]:
d['issue_cr_diff_mnth'] = [int(x) for x in d['issue_cr_diff'] / np.timedelta64(1, 'M')]

Let us imagine that the current date when we are calculating the risk is 2020-01-01.

In [25]:
cur_date = datetime.datetime(2020, 1, 1)

# Creating an additional column to see days past since the loan issue 
d['days_since_issue'] = cur_date - d['issue_d_date']
d['mnth_since_issue'] = [int(x) for x in d['days_since_issue'] / np.timedelta64(1, 'M')]

In [26]:
# Inspecting the results of engineering
d[[
    'earliest_cr_line', 
    'earliest_cr_line_date', 
    'issue_d', 
    'issue_d_date', 
    'issue_cr_diff',
    'issue_cr_diff_mnth',
    'mnth_since_issue'
]].sample(10)

,earliest_cr_line,earliest_cr_line_date,issue_d,issue_d_date,issue_cr_diff,issue_cr_diff_mnth,mnth_since_issue
2719167,Jan-1994,1994-01-01,Mar-2017,2017-03-01,8460 days,277,34
489050,Jun-2000,2000-06-01,Mar-2018,2018-03-01,6482 days,212,22
34477,Jun-1983,1983-06-01,Oct-2009,2009-10-01,9619 days,316,123
2902799,Oct-1988,1988-10-01,Apr-2017,2017-04-01,10409 days,341,33
2437949,Mar-2001,2001-03-01,Jun-2016,2016-06-01,5571 days,183,43
2551333,Jun-1988,1988-06-01,Aug-2016,2016-08-01,10288 days,338,41
1650442,Nov-2001,2001-11-01,May-2014,2014-05-01,4564 days,149,68
33687,Jun-1997,1997-06-01,Dec-2009,2009-12-01,4566 days,150,121
2754620,Oct-1993,1993-10-01,Feb-2017,2017-02-01,8524 days,280,34
942663,Sep-2013,2013-09-01,Oct-2018,2018-10-01,1856 days,60,15


## Preprocesing the categorical variables 

The aim of categorical variable preprocesing is to convert them into dummy variables. 

In [45]:
categorical_vars = [
    'grade',
    'sub_grade',
    'home_ownership',
    'verification_status',
    'purpose',
    'addr_state',
    'initial_list_status',
    'emp_length'
]

dummylist = []

for categorical_var in categorical_vars:
    dummylist.append(pd.get_dummies(d[categorical_var], prefix=f'{categorical_var}'))
    
# Creating the dummy dataframe for the categorical variables
dummydf = pd.concat(dummylist, axis=1)

In [28]:
# Listing all the columns
dummydf.columns.values

array(['grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'sub_grade_A1', 'sub_grade_A2', 'sub_grade_A3',
       'sub_grade_A4', 'sub_grade_A5', 'sub_grade_B1', 'sub_grade_B2',
       'sub_grade_B3', 'sub_grade_B4', 'sub_grade_B5', 'sub_grade_C1',
       'sub_grade_C2', 'sub_grade_C3', 'sub_grade_C4', 'sub_grade_C5',
       'sub_grade_D1', 'sub_grade_D2', 'sub_grade_D3', 'sub_grade_D4',
       'sub_grade_D5', 'sub_grade_E1', 'sub_grade_E2', 'sub_grade_E3',
       'sub_grade_E4', 'sub_grade_E5', 'sub_grade_F1', 'sub_grade_F2',
       'sub_grade_F3', 'sub_grade_F4', 'sub_grade_F5', 'sub_grade_G1',
       'sub_grade_G2', 'sub_grade_G3', 'sub_grade_G4', 'sub_grade_G5',
       'home_ownership_ANY', 'home_ownership_MORTGAGE',
       'home_ownership_NONE', 'home_ownership_OTHER',
       'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Not Verified',
       'verification_status_Source Verified',
       'verification_status_Verified', 'pu

In [29]:
# Getting the numeric feature list 
numeric_features = [
    'loan_amnt',
    'funded_amnt',
    'term',
    'emp_length',
    'annual_inc',
    'mnth_since_issue',
    'term_int',
    # Adding load_status for Y creation
    'loan_status'
]

In [30]:
# Creating the dataframe with numeric and dummy variables 
d = pd.concat([d, dummydf], axis=1)

# Probability of default (PD) modeling

The main objective in PD modeling is to create a model to infer a probability for a default given a set of features $X$.

It is established by various auditing firms that the interpretation of the model needs to be easy even for not statistical savy people. One highly interpretable machine learning model is logistic regression. 

Another agreed upon part of PD modeling is that the $X$ matrix needs to be comprised of dummy variable data. Thus, even continues numeric variables need to be converted to categorical features. Additionally, when creating the $Y$ variable, positive coefficients in logistic regression need to lead to a "positive" class, meaning, the higher a given value, the less chance of default.

# Creation of the dependant variable 

One of the widespread default definition is that if a person is not able to pay their obligations in 90 days, it is considered default. In the PD modeling space, the Y encoding to non defaul and default is often termed "good bad" encoding.

In [31]:
d['loan_status'].value_counts() / d['loan_status'].count()

Fully Paid                                             0.511938
Current                                                0.352573
Charged Off                                            0.123825
Late (31-120 days)                                     0.005522
In Grace Period                                        0.003430
Late (16-30 days)                                      0.000929
Issued                                                 0.000706
Does not meet the credit policy. Status:Fully Paid     0.000670
Does not meet the credit policy. Status:Charged Off    0.000259
Default                                                0.000148
Name: loan_status, dtype: float64

In [32]:
# Creating the Y (good_bad) variable for the probability of default analysis
d['good_bad'] = np.where(d['loan_status'].isin([
    'Charged Off', 
    'Default',
    'Late (31-120 days)',
    'Does not meet the credit policy. Status:Charged Off'
]), 0, 1)

In [33]:
# Distribution of classes
d['good_bad'].value_counts() / d['good_bad'].count()

1    0.870246
0    0.129754
Name: good_bad, dtype: float64

# Spliting the data into train, validation and test sets

The amount of data for training will be 80 percent of the whole dataset, validation (used for hyperparameter tuning) will be 10 percent and the test set (which we will measure some accuracy metrics) will be 10 percent. 

In [34]:
train, val, test = np.split(d.sample(frac=1, random_state=42), [int(.8*len(d)), int(.9*len(d))])

In [35]:
print(f"Shape of training data: {train.shape}")
print(f"Shape of training data: {val.shape}")
print(f"Shape of training data: {test.shape}")

Shape of training data: (2335644, 146)
Shape of training data: (291956, 146)
Shape of training data: (291956, 146)


# Fine classing, weight of evidence and coarse classing

Fine classing is a technique that groups a variable's values into a number of fine bins. Using these bins, a measure of the variable's predictive power, known as information value (IV), can be computed. Also from these fine bins, further grouping can be carried out to result in coarse classing.

One of the fundamental concepts in information value is the weight of evidence concept. The weight of evidence tells the predictive power of an independent variable in relation to the dependent variable.

For a given feature $i$, the weight of evidence, or, $WoE_{i}$ is calculated by: 

$$ WoE_{i} = log\left(\dfrac{\% of y = 1 | x = i}{\% of y = 0 | x = i}\right)$$

Thus, the woe is calculated for each feature in our dataset.

## Caluclating the WoE for home ownership

Lets go step by step and calculate the WoE statistic for all the levels of the home ownership categorical value. 

In [36]:
# Subseting the data
woe = d[['home_ownership', 'good_bad']]

# Lets calculate the number of borrowers for each status 
grouped = woe.groupby('home_ownership', as_index=False)['good_bad'].count()

In [37]:
# Inspecting the total number of people in each group 
grouped

,home_ownership,good_bad
0,ANY,3515
1,MORTGAGE,1434863
2,NONE,51
3,OTHER,181
4,OWN,329907
5,RENT,1151039


In [38]:
# Getting the total good borrowers in each group 
groupedgood = woe.groupby('home_ownership', as_index=False)['good_bad'].sum().rename(columns={'good_bad': 'good_borrowers'})

# Merging with the initial dataframe
grouped = pd.merge(grouped, groupedgood, on='home_ownership')

# Getting the number of bad borrowers
grouped['bad_borrowers'] = grouped['good_bad'] - grouped['good_borrowers']

# Calculating the share of good borrowers
grouped['share_good'] = grouped['good_borrowers'] / grouped['good_bad']

# Calculating the share of bad borrowers
grouped['share_bad'] = 1 - grouped['share_good']

# Seeing the results
grouped

,home_ownership,good_bad,good_borrowers,bad_borrowers,share_good,share_bad
0,ANY,3515,3225,290,0.917496,0.082504
1,MORTGAGE,1434863,1273482,161381,0.887529,0.112471
2,NONE,51,43,8,0.843137,0.156863
3,OTHER,181,143,38,0.790055,0.209945
4,OWN,329907,286656,43251,0.868899,0.131101
5,RENT,1151039,977184,173855,0.848958,0.151042


In [39]:
# Calculating the total proportion of good/bad borrowers in each group 
grouped['prop_n_good'] = grouped['good_borrowers'] / grouped['good_borrowers'].sum()
grouped['prop_n_bad'] = grouped['bad_borrowers'] / grouped['bad_borrowers'].sum()

grouped

,home_ownership,good_bad,good_borrowers,bad_borrowers,share_good,share_bad,prop_n_good,prop_n_bad
0,ANY,3515,3225,290,0.917496,0.082504,0.001269,0.000766
1,MORTGAGE,1434863,1273482,161381,0.887529,0.112471,0.501226,0.426006
2,NONE,51,43,8,0.843137,0.156863,0.000017,0.000021
3,OTHER,181,143,38,0.790055,0.209945,0.000056,0.000100
4,OWN,329907,286656,43251,0.868899,0.131101,0.112824,0.114172
5,RENT,1151039,977184,173855,0.848958,0.151042,0.384607,0.458935


In the above dataset we have everything we need to calculate the weight of evidence for each categorical variable level. 

In [40]:
grouped['woe'] = np.log(grouped['prop_n_good'] / grouped['prop_n_bad'])

In [41]:
grouped

,home_ownership,good_bad,good_borrowers,bad_borrowers,share_good,share_bad,prop_n_good,prop_n_bad,woe
0,ANY,3515,3225,290,0.917496,0.082504,0.001269,0.000766,0.505668
1,MORTGAGE,1434863,1273482,161381,0.887529,0.112471,0.501226,0.426006,0.162603
2,NONE,51,43,8,0.843137,0.156863,0.000017,0.000021,-0.221380
3,OTHER,181,143,38,0.790055,0.209945,0.000056,0.000100,-0.577880
4,OWN,329907,286656,43251,0.868899,0.131101,0.112824,0.114172,-0.011876
5,RENT,1151039,977184,173855,0.848958,0.151042,0.384607,0.458935,-0.176685


In [42]:
# Lets calculate the IV for the overall variable home_ownership
IV = np.sum((grouped['prop_n_good'] - grouped['prop_n_bad']) * grouped['woe'])

print(f'The information value is :{IV}')

The information value is :0.025660726093582335


## Defining a function that calculates the woe and IV 

In [43]:
def get_woe(d: pd.DataFrame, cat_var: str, good_bad_var: str):
    # Subseting the dataframe 
    d = d[[cat_var, good_bad_var]]
    
    # Calculating the number of borrowers for each status 
    grouped = d.groupby(cat_var, as_index=False)[good_bad_var].count().rename(columns={good_bad_var: "n"})
    
    groupedgood = d.groupby(cat_var, as_index=False)[good_bad_var].sum().rename(columns={good_bad_var: 'good_borrowers'})

    # Merging with the initial dataframe
    grouped = pd.merge(grouped, groupedgood, on=cat_var)

    # Getting the number of bad borrowers
    grouped['bad_borrowers'] = grouped['n'] - grouped['good_borrowers']

    # Calculating the share of good borrowers
    grouped['share_good'] = grouped['good_borrowers'] / grouped['n']

    # Calculating the share of bad borrowers
    grouped['share_bad'] = 1 - grouped['n']
    
    # Calculating the total proportion of good/bad borrowers in each group 
    grouped['prop_n_good'] = grouped['good_borrowers'] / grouped['good_borrowers'].sum()
    grouped['prop_n_bad'] = grouped['bad_borrowers'] / grouped['bad_borrowers'].sum()
    
    # Calculating the WoE statistic
    grouped['woe'] = np.log(grouped['prop_n_good'] / grouped['prop_n_bad'])
    
    # Sorting by woe
    grouped.sort_values('woe', inplace=True, ascending=False)
    grouped.reset_index(inplace=True, drop=True)
    
    # Calculating IV
    grouped['IV'] = np.sum((grouped['prop_n_good'] - grouped['prop_n_bad']) * grouped['woe'])
    
    # Returning the aggregated dataframe
    return grouped

In [46]:
# Printing out the frame for all the categorical variables
for var in categorical_vars:
    print('-----------')
    print(f'{var}:')
    print(get_woe(d, var, 'good_bad'))
    print('-----------')

-----------
grade:
  grade       n  good_borrowers  bad_borrowers  share_good  share_bad  \
0     A  654650          630541          24109    0.963173    -654649   
1     B  855747          777869          77878    0.908994    -855746   
2     C  800807          677416         123391    0.845917    -800806   
3     D  415593          328934          86659    0.791481    -415592   
4     E  138808           94090          44718    0.677843    -138807   
5     F   41781           25071          16710    0.600057     -41780   
6     G   12170            6812           5358    0.559737     -12169   

   prop_n_good  prop_n_bad       woe        IV  
0     0.248173    0.063642  1.360854  0.523789  
1     0.306159    0.205579  0.398276  0.523789  
2     0.266622    0.325722 -0.200211  0.523789  
3     0.129464    0.228759 -0.569263  0.523789  
4     0.037033    0.118045 -1.159263  0.523789  
5     0.009868    0.044110 -1.497434  0.523789  
6     0.002681    0.014144 -1.663044  0.523789  
----

   addr_state       n  good_borrowers  bad_borrowers  share_good  share_bad  \
0          ME    7237            6738            499    0.931049      -7236   
1          VT    6499            5959            540    0.916910      -6498   
2          WV   12265           11233           1032    0.915858     -12264   
3          ID    6796            6216            580    0.914656      -6795   
4          OR   35229           31905           3324    0.905646     -35228   
5          DC    6724            6083            641    0.904670      -6723   
6          NH   14516           13088           1428    0.901626     -14515   
7          SC   36776           33061           3715    0.898983     -36775   
8          ND    5027            4501            526    0.895365      -5026   
9          WA   61261           54831           6430    0.895039     -61260   
10         CO   61134           54683           6451    0.894478     -61133   
11         MT    8202            7317            885

Based on the WoE statistic we need to figure out how to organize and lump together the original discrete features.